# Clustering Crypto

In [29]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [2]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [3]:
# # Create a DataFrame 
# data = requests.get(url).json()
# df = pd.DataFrame(data['Data']).T

# df.head()

In [4]:
# Alternatively, use the provided csv file:
file_path = f'../data/crypto_data.csv'
crypto_df = pd.read_csv(file_path, index_col = 0)

crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [23]:
# Keep only necessary columns:
crypto_columns = ['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']
crypto_df_clean = crypto_df[crypto_columns]

print(crypto_df_clean.shape)
crypto_df_clean.head()

(1252, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [24]:
# Keep only cryptocurrencies that are trading
crypto_df_clean = crypto_df_clean.loc[crypto_df_clean['IsTrading'] == True, :]

print(crypto_df_clean.shape)
crypto_df_clean.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [25]:
# Keep only cryptocurrencies with a working algorithm
crypto_df_clean = crypto_df_clean[crypto_df_clean['Algorithm'].notna()]

print(crypto_df_clean.shape)
crypto_df_clean.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [26]:
# Remove the "IsTrading" column
crypto_df_clean = crypto_df_clean.drop(columns = ['IsTrading'])

crypto_df_clean.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [27]:
# Remove rows with at least 1 null value
crypto_df_clean = crypto_df_clean.dropna()

print(crypto_df_clean.shape)
crypto_df_clean.head()

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [28]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df_clean = crypto_df_clean.loc[crypto_df_clean['TotalCoinsMined'] != 0]

print(crypto_df_clean.shape)
crypto_df_clean.head()

(533, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [33]:
# Drop rows where there are 'N/A' text values
crypto_df_clean = crypto_df_clean[(crypto_df_clean != 'N/A').all(axis =1)]

print(crypto_df_clean.shape)
crypto_df_clean.head()

(533, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [34]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(crypto_df_clean['CoinName'], index = crypto_df_clean.index)

print(coins_name.shape)
coins_name.head()

(533, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [36]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df_clean = crypto_df_clean.drop(columns = ['CoinName'])

print(crypto_df_clean.shape)
crypto_df_clean.head()

(533, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [45]:
# Create dummy variables for text features
crypto_df_clean['TotalCoinsMined'] = crypto_df_clean['TotalCoinsMined'].apply(lambda x: float(x))
crypto_df_clean['TotalCoinSupply'] = crypto_df_clean['TotalCoinSupply'].apply(lambda x: float(x))

X = pd.get_dummies(crypto_df_clean, columns = ['Algorithm', 'ProofType'])

print(X.shape)
X.head()

(533, 100)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)

print(X_scaled[0])

[-0.11674788 -0.15286468 -0.0433555  -0.0433555  -0.0433555  -0.06137164
 -0.07523548 -0.0433555  -0.06137164 -0.06137164 -0.0433555  -0.0433555
 -0.19226279 -0.06137164 -0.09731237 -0.0433555  -0.11536024 -0.07523548
 -0.0433555  -0.0433555  -0.15176505 -0.0433555  -0.13105561 -0.0433555
 -0.0433555  -0.08695652 -0.0433555  -0.0433555  -0.0433555  -0.0433555
 -0.06137164 -0.0433555  -0.08695652 -0.08695652 -0.08695652 -0.0433555
 -0.13105561 -0.13827675 -0.13827675 -0.0433555  -0.06137164 -0.0433555
 -0.07523548 -0.1815096  -0.0433555  -0.0433555  -0.0433555  -0.07523548
 -0.15811388 -0.3145935  -0.0433555  -0.08695652 -0.07523548 -0.06137164
 -0.0433555   1.38873015 -0.0433555  -0.0433555  -0.06137164 -0.0433555
 -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
 -0.0433555  -0.39836623 -0.0433555  -0.1815096  -0.0433555  -0.08695652
 -0.08695652 -0.10670145 -0.0433555  -0.0433555  -0.13105561 -0.0433555
 -0.0433555  -0.0433555  -0.0433555  -0.07523548 -0.4386271

### Reducing Dimensions Using PCA

In [50]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components = 3)

X_pca = pca.fit_transform(X_scaled)

In [52]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(X_pca, index = crypto_df_clean.index, columns = ['pc_1', 'pc_2', 'pc_3'])

pcs_df.head(10)

,pc_1,pc_2,pc_3
42,-0.333229,1.007540,-0.534093
404,-0.316504,1.007560,-0.534332
1337,2.310063,1.622218,-0.533869
BTC,-0.152170,-1.272462,0.169381
ETH,-0.158449,-1.981389,0.306344
LTC,-0.163761,-1.126841,-0.061794
DASH,-0.391493,1.193867,-0.362269
XMR,-0.167401,-2.181933,0.291203
ETC,-0.156886,-1.981499,0.306331
ZEC,-0.129335,-2.128327,0.435802


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [54]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters = i, random_state = 14)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {'k' : k, 'inertia' : inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x = 'k', y = 'inertia', xticks = k, title = 'elbow_curve')

:Curve   [k]   (inertia)

Running K-Means with `k = 5`

In [87]:
# Initialize the K-Means model
model = KMeans(n_clusters = 5, random_state = 14)

# Fit the model
model.fit(pcs_df)

# Predict clusters
prediction = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df['class'] = model.labels_

clustered_df = pd.concat([coins_name, crypto_df_clean, pcs_df], join = 'inner', axis = 1)

print(clustered_df.shape)
clustered_df.head()

(533, 9)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc_1,pc_2,pc_3,class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.333229,1.007540,-0.534093,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.316504,1.007560,-0.534332,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.310063,1.622218,-0.533869,4
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.152170,-1.272462,0.169381,1
ETH,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00,-0.158449,-1.981389,0.306344,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [89]:
# Create a 3D-Scatter with the PCA data and the clusters
clustered_fig = px.scatter_3d(
    clustered_df,
    x = 'pc_3',
    y = 'pc_2',
    z = 'pc_1',
    color = 'class',
    symbol = 'class',
    hover_name = 'CoinName',
    hover_data = ['Algorithm'],
    width = 800
)
clustered_fig.update_layout(legend = dict(x = 0, y = 1))
clustered_fig.show()

#### Table of Tradable Cryptocurrencies

In [90]:
# Table with tradable cryptos
clustered_table = clustered_df.hvplot.table(columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'], sortable = True, selectable = True)

clustered_table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [84]:
# Print the total number of tradable cryptocurrencies
print(len(clustered_df['CoinName'].unique()))

532


#### Scatter Plot with Tradable Cryptocurrencies

In [115]:
# Scale data to create the scatter plot
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'] / 1000000000
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'] / 1000000000

In [123]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"

clustered_scatter = clustered_df.hvplot.scatter(
    x = 'TotalCoinsMined',
    y = 'TotalCoinSupply',
    hover_cols = ['CoinName'],
    by = 'class',
    xlabel = 'total_coins_mined_(billions)',
    ylabel = 'total_coins_supply_(billions)',
)

clustered_scatter

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

In [117]:
hvplot.help('scatter')


Scatter plot

Parameters
----------
x, y : string, optional
    Field name to draw x- and y-positions from
c: string, optional
    Name of the field to color points by
s: string, optional
    Name of the field to scale point size by
scale: number, optional
    Scaling factor to apply to point scaling
**kwds : optional
    Keyword arguments to pass on to
    :py:meth:`hvplot.converter.HoloViewsConverter`.
Returns
-------
obj : HoloViews object
    The HoloViews representation of the plot.


Generic options
---------------
colorbar (default=False): boolean
    Enables colorbar
fontscale: number
    Scales the size of all fonts by the same amount, e.g. fontscale=1.5
    enlarges all fonts (title, xticks, labels etc.) by 50%
fontsize: number or dict
    Set title, label and legend text to the same fontsize. Finer control
    by using a dict: {'title': '15pt', 'ylabel': '5px', 'ticks': 20}
flip_xaxis/flip_yaxis: boolean
    Whether to flip the axis left to right or up and down respectively